In [21]:
from vega_datasets import data
import altair as alt
import pandas as pd

employment_data = data.us_employment()

employment_data.total = employment_data.sum(numeric_only=True)

employment_data.month = pd.to_datetime(employment_data.month)

employment_data['year'] = pd.DatetimeIndex(employment_data.month).year

employment_long = pd.melt(employment_data, id_vars=['month', 'year'], value_name='totalfigure').reset_index()

employment_long.rename(columns={'variable':'employment_type'}, inplace=True)

employment_long.head(50)


# employment_data.pivot(
#     columns=[
#         "month",
#                 "nonfarm",
#         "private",
#         "goods_producing",
#         "service_providing",
#         "private_service_providing",
#         "mining_and_logging",
#         "construction",
#         "manufacturing",
#         "durable_goods",
#         "nondurable_goods",
#         "trade_transportation_utilties",
#         "wholesale_trade",
#         "retail_trade",
#         "transportation_and_warehousing",
#         "utilities",
#         "information",
#         "financial_activities",
#         "professional_and_business_services",
#         "education_and_health_services",
#         "leisure_and_hospitality",
#         "other_services",
#         "government",
#         "nonfarm_change",
#         ],
#     values=[

#         'total'
#     ]
# ).reset_index()


# employment_data.to_csv('employment_data.csv')



C:\Users\Chris Henry\AppData\Local\Temp\ipykernel_13920\577360768.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  employment_data.total = employment_data.sum(numeric_only=True)


,index,month,year,employment_type,totalfigure
0,0,2006-01-01,2006,nonfarm,135450.0
1,1,2006-02-01,2006,nonfarm,135762.0
2,2,2006-03-01,2006,nonfarm,136059.0
3,3,2006-04-01,2006,nonfarm,136227.0
4,4,2006-05-01,2006,nonfarm,136258.0
5,5,2006-06-01,2006,nonfarm,136337.0
6,6,2006-07-01,2006,nonfarm,136543.0
7,7,2006-08-01,2006,nonfarm,136722.0
8,8,2006-09-01,2006,nonfarm,136870.0
9,9,2006-10-01,2006,nonfarm,136882.0


In [14]:
slider = alt.binding_range(min=min(employment_long['year']), max=max(employment_long['year']), step=1, name='Year')
selector = alt.param(name='YearSlicer', value = 2006, bind=slider)

alt.Chart(employment_long).mark_area().encode(
    x='month:T',
    y='totalfigure:Q',
    color="employment_type:N",
    tooltip=['employment_type','totalfigure', 'month:T']
    
    ).properties(
        width=1800,
        height=1200
    ).add_params(selector)#.transform_timeunit(month='month(month)')

alt.Chart(...)

In [31]:

alt.Chart(employment_long).mark_area().encode(
    x='month:T',
    y=alt.Y('totalfigure:Q',).stack('normalize'),
    color="employment_type:N",
    tooltip=['employment_type','totalfigure', 'month:T']
    
    ).properties(
        width=1800,
        height=1200
    )

alt.Chart(...)

In [18]:
slider = alt.binding_range(min=min(employment_long['year']), max=max(employment_long['year']), step=1, name='Year')
selector = alt.param(name='YearSlicer', value = 2006, bind=slider)

alt.Chart(employment_long).mark_bar().encode(
    x='year:N',
    y='totalfigure:Q',
    color="employment_type:N",
    tooltip=['employment_type','totalfigure', 'month:T']
    
    ).properties(
        width=1800,
        height=1200
    ).add_params(selector)

alt.Chart(...)

In [34]:

alt.Chart(employment_long).mark_bar().encode(
    x='year:N',
    y=alt.Y('totalfigure:Q',).stack('normalize'),
    color="employment_type:N",
    tooltip=['employment_type','totalfigure', 'month:T']
    
    ).properties(
        width=1800,
        height=1200
    )

alt.Chart(...)

In [26]:
alt.Chart(employment_long).mark_boxplot().encode(
    
        x='year:N',
    y='totalfigure:Q',
    color="employment_type:N",
    tooltip=['employment_type','totalfigure', 'month:T']
).properties(
        width=1800,
        height=1200
    )

alt.Chart(...)

In [28]:
alt.Chart(employment_long).mark_point().encode(
    
        x='mean(totalfigure):Q',
    y='totalfigure:Q',
    color="employment_type:N",
    tooltip=['employment_type','totalfigure', 'month:T']
).properties(
        width=1800,
        height=1200
    )

alt.Chart(...)